# Initialisation 

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/Users/chanmunfai/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## Pre-processsing 

Here, we will remove all missing values, convert datetime into a numeric form and extract feautures from it, convert count into a logarithmic form and drop columns 'casual' and 'registered' as these are unavailable on our test set. 

We will not have to re-run this everytime as we will save the pre-processed train file into a feather format. 

In [4]:
train_features = pd.read_csv("/Users/chanmunfai/fastai/courses/ml1/Dengue/dengue_features_train.csv", low_memory = False)

In [6]:
train_labels = pd.read_csv("/Users/chanmunfai/fastai/courses/ml1/Dengue/dengue_labels_train.csv", low_memory = False)

In [8]:
train_features.shape, train_labels.shape

((1456, 24), (1456, 4))

In [10]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [13]:
display_all(train_features.tail().T)

,1451,1452,1453,1454,1455
city,iq,iq,iq,iq,iq
year,2010,2010,2010,2010,2010
weekofyear,21,22,23,24,25
week_start_date,2010-05-28,2010-06-04,2010-06-11,2010-06-18,2010-06-25
ndvi_ne,0.34275,0.160157,0.247057,0.333914,0.298186
ndvi_nw,0.3189,0.160371,0.146057,0.245771,0.232971
ndvi_se,0.256343,0.136043,0.250357,0.278886,0.274214
ndvi_sw,0.292514,0.225657,0.233714,0.325486,0.315757
precipitation_amt_mm,55.3,86.47,58.94,59.67,63.22
reanalysis_air_temp_k,299.334,298.33,296.599,296.346,298.097


In [14]:
display_all(train_labels.tail().T)

,1451,1452,1453,1454,1455
city,iq,iq,iq,iq,iq
year,2010,2010,2010,2010,2010
weekofyear,21,22,23,24,25
total_cases,5,8,1,1,4


In [15]:
train_labels.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [37]:
train = pd.concat([train_features, train_labels], axis = 1)

In [42]:
train.shape

(1456, 25)

In [40]:
train = train.loc[:,~train.columns.duplicated()]

In [78]:
display_all(train.head().T)

,0,1,2,3,4
city,sj,sj,sj,sj,sj
year,1990,1990,1990,1990,1990
weekofyear,18,19,20,21,22
ndvi_ne,0.1226,0.1699,0.03225,0.128633,0.1962
ndvi_nw,0.103725,0.142175,0.172967,0.245067,0.2622
ndvi_se,0.198483,0.162357,0.1572,0.227557,0.2512
ndvi_sw,0.177617,0.155486,0.170843,0.235886,0.24734
precipitation_amt_mm,12.42,22.82,34.54,15.36,7.52
reanalysis_air_temp_k,297.573,298.211,298.781,298.987,299.519
reanalysis_avg_temp_k,297.743,298.443,298.879,299.229,299.664


### Removing Missing Values

In [43]:
display_all(train.isnull().sum().sort_index()/len(train))

city                                     0.000000
ndvi_ne                                  0.133242
ndvi_nw                                  0.035714
ndvi_se                                  0.015110
ndvi_sw                                  0.015110
precipitation_amt_mm                     0.008929
reanalysis_air_temp_k                    0.006868
reanalysis_avg_temp_k                    0.006868
reanalysis_dew_point_temp_k              0.006868
reanalysis_max_air_temp_k                0.006868
reanalysis_min_air_temp_k                0.006868
reanalysis_precip_amt_kg_per_m2          0.006868
reanalysis_relative_humidity_percent     0.006868
reanalysis_sat_precip_amt_mm             0.008929
reanalysis_specific_humidity_g_per_kg    0.006868
reanalysis_tdtr_k                        0.006868
station_avg_temp_c                       0.029533
station_diur_temp_rng_c                  0.029533
station_max_temp_c                       0.013736
station_min_temp_c                       0.009615


In [45]:
train.dtypes

city                                      object
year                                       int64
weekofyear                                 int64
week_start_date                           object
ndvi_ne                                  float64
ndvi_nw                                  float64
ndvi_se                                  float64
ndvi_sw                                  float64
precipitation_amt_mm                     float64
reanalysis_air_temp_k                    float64
reanalysis_avg_temp_k                    float64
reanalysis_dew_point_temp_k              float64
reanalysis_max_air_temp_k                float64
reanalysis_min_air_temp_k                float64
reanalysis_precip_amt_kg_per_m2          float64
reanalysis_relative_humidity_percent     float64
reanalysis_sat_precip_amt_mm             float64
reanalysis_specific_humidity_g_per_kg    float64
reanalysis_tdtr_k                        float64
station_avg_temp_c                       float64
station_diur_temp_rn

In [46]:
add_datepart(train, 'week_start_date')

In [47]:
train.dtypes

city                                      object
year                                       int64
weekofyear                                 int64
ndvi_ne                                  float64
ndvi_nw                                  float64
ndvi_se                                  float64
ndvi_sw                                  float64
precipitation_amt_mm                     float64
reanalysis_air_temp_k                    float64
reanalysis_avg_temp_k                    float64
reanalysis_dew_point_temp_k              float64
reanalysis_max_air_temp_k                float64
reanalysis_min_air_temp_k                float64
reanalysis_precip_amt_kg_per_m2          float64
reanalysis_relative_humidity_percent     float64
reanalysis_sat_precip_amt_mm             float64
reanalysis_specific_humidity_g_per_kg    float64
reanalysis_tdtr_k                        float64
station_avg_temp_c                       float64
station_diur_temp_rng_c                  float64
station_max_temp_c  

In [71]:
display_all(train.tail().T)

,1451,1452,1453,1454,1455
city,iq,iq,iq,iq,iq
year,2010,2010,2010,2010,2010
weekofyear,21,22,23,24,25
ndvi_ne,0.34275,0.160157,0.247057,0.333914,0.298186
ndvi_nw,0.3189,0.160371,0.146057,0.245771,0.232971
ndvi_se,0.256343,0.136043,0.250357,0.278886,0.274214
ndvi_sw,0.292514,0.225657,0.233714,0.325486,0.315757
precipitation_amt_mm,55.3,86.47,58.94,59.67,63.22
reanalysis_air_temp_k,299.334,298.33,296.599,296.346,298.097
reanalysis_avg_temp_k,300.771,299.393,297.593,297.521,299.836


This is an illegal match, so we remove it. 

In [51]:
train.drop(['week_start_Year', 'week_start_Week'], axis =1, inplace = True)

In [56]:
train.dtypes

city                                     category
year                                        int64
weekofyear                                  int64
ndvi_ne                                   float64
ndvi_nw                                   float64
ndvi_se                                   float64
ndvi_sw                                   float64
precipitation_amt_mm                      float64
reanalysis_air_temp_k                     float64
reanalysis_avg_temp_k                     float64
reanalysis_dew_point_temp_k               float64
reanalysis_max_air_temp_k                 float64
reanalysis_min_air_temp_k                 float64
reanalysis_precip_amt_kg_per_m2           float64
reanalysis_relative_humidity_percent      float64
reanalysis_sat_precip_amt_mm              float64
reanalysis_specific_humidity_g_per_kg     float64
reanalysis_tdtr_k                         float64
station_avg_temp_c                        float64
station_diur_temp_rng_c                   float64


In [55]:
train_cats(train)

In [57]:
train.city.cat.categories

Index(['iq', 'sj'], dtype='object')

In [59]:
os.makedirs('tmp', exist_ok=True)
train.to_feather('tmp/dengue-raw')

## Running a Basic Random Forest Model 

I would want to examine if random forest models are able to extrapolate to time series models. 

In [60]:
df_raw = pd.read_feather('tmp/dengue-raw')

In [61]:
df, y, nas = proc_df(df_raw, 'total_cases')

In [62]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.9719622243134957

Split into training and validation set 

In [65]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 200  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((1256, 54), (1256,), (200, 54), (200,))

In [66]:
# Metric used for the DengAI competition (Mean Absolute Error (MAE))
from sklearn.metrics import mean_absolute_error

# Function to print the MAE (Mean Absolute Error) score
# This is the metric used by Kaggle in this competition
def print_score(m : RandomForestRegressor):
    res = ['mae train: ', mean_absolute_error(m.predict(X_train), y_train), 
           'mae val: ', mean_absolute_error(m.predict(X_valid), y_valid),
           'R square train:', m.score(X_train, y_train), 
           'R square val:', m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [67]:
m1 = RandomForestRegressor(n_jobs=-1)
m1.fit(X_train, y_train)
print_score(m1)

['mae train: ', 3.939028662420382, 'mae val: ', 7.4075999999999995, 'R square train:', 0.9725653082995526, 'R square val:', -0.10007350455069242]


Clearly, there is some problem with our validation set as we have only selected 'iq' but not 'sf'.

## Redefining our validation set 

Let us first look at our test set, so that our validation set can resemble it. 

In [101]:
test = pd.read_csv("/Users/chanmunfai/fastai/courses/ml1/Dengue/dengue_features_test.csv", low_memory = False)

In [120]:
test.shape

(416, 24)

In [108]:
test[test['city'] == 'sj'].tail()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
255,sj,2013,13,2013-03-26,-0.08740,-0.016183,0.156343,0.105186,30.34,298.670000,...,2.55,78.780000,30.34,15.985714,3.314286,27.542857,7.942857,33.9,22.8,3.5
256,sj,2013,14,2013-04-02,-0.20325,-0.077833,0.204171,0.178914,6.55,298.035714,...,64.30,81.650000,6.55,15.881429,2.828571,26.642857,6.642857,33.3,22.8,17.6
257,sj,2013,15,2013-04-09,-0.11760,-0.008200,0.192700,0.170429,0.00,299.057143,...,0.70,78.285714,0.00,16.212857,3.171429,27.914286,8.114286,32.8,23.3,9.4
258,sj,2013,16,2013-04-16,0.08275,0.031200,0.135014,0.074857,0.00,298.912857,...,1.40,77.674286,0.00,15.965714,3.042857,27.728571,6.942857,31.7,23.9,22.9
259,sj,2013,17,2013-04-23,-0.08730,-0.048667,0.129814,0.117671,45.47,298.067143,...,19.90,79.045714,45.47,15.451429,2.342857,26.442857,6.742857,31.1,21.7,47.5


In [109]:
display(test[test['city'] == 'sj'].shape), display(test[test['city'] == 'iq'].shape)

(260, 24)

(156, 24)

(None, None)

In [115]:
260/ (260 + 156) 

0.625

In [80]:
display(train[train['city'] == 'sj'].shape), display(train[train['city'] == 'iq'].shape)

(936, 35)

(520, 35)

(None, None)

In [117]:
0.625 * 0.2 * len(train)

182.0

In [118]:
(1-0.625) * 0.2 * len(train)

109.20000000000002

Our test set requires us to test our model on the most recent dates, so we need our validation set to be on the most recent dates as well. 

We may want to experiment with different months and weeks of year etc in future validation sets, but this will do for now. 

Also, we have 62.5% of our test set on 'sj' cities, so if we want a 20% validation set that equates to 182 'sj' datapoints on validation and 110 'iq' validation datapoints. 

#### Resplitting the validation sets

In [133]:
train_sj = train[train['city'] == 'sj']; 
train_sj.tail()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,week_start_Day,week_start_Dayofweek,week_start_Dayofyear,week_start_Is_month_end,week_start_Is_month_start,week_start_Is_quarter_end,week_start_Is_quarter_start,week_start_Is_year_end,week_start_Is_year_start,week_start_Elapsed
931,sj,2008,13,0.07785,-0.039900,0.310471,0.296243,27.19,296.958571,296.957143,...,25,1,85,False,False,False,False,False,False,1206403200
932,sj,2008,14,-0.03800,-0.016833,0.119371,0.066386,3.82,298.081429,298.228571,...,1,1,92,False,True,False,True,False,False,1207008000
933,sj,2008,15,-0.15520,-0.052750,0.137757,0.141214,16.96,297.460000,297.564286,...,8,1,99,False,False,False,False,False,False,1207612800
934,sj,2008,16,0.00180,NaN,0.203900,0.209843,0.00,297.630000,297.778571,...,15,1,106,False,False,False,False,False,False,1208217600
935,sj,2008,17,-0.03700,-0.010367,0.077314,0.090586,0.00,298.672857,298.692857,...,22,1,113,False,False,False,False,False,False,1208822400


In [135]:
train_iq = train[train['city'] == 'iq']; 
train_iq.tail()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,week_start_Day,week_start_Dayofweek,week_start_Dayofyear,week_start_Is_month_end,week_start_Is_month_start,week_start_Is_quarter_end,week_start_Is_quarter_start,week_start_Is_year_end,week_start_Is_year_start,week_start_Elapsed
1451,iq,2010,21,0.342750,0.318900,0.256343,0.292514,55.30,299.334286,300.771429,...,28,4,148,False,False,False,False,False,False,1275004800
1452,iq,2010,22,0.160157,0.160371,0.136043,0.225657,86.47,298.330000,299.392857,...,4,4,155,False,False,False,False,False,False,1275609600
1453,iq,2010,23,0.247057,0.146057,0.250357,0.233714,58.94,296.598571,297.592857,...,11,4,162,False,False,False,False,False,False,1276214400
1454,iq,2010,24,0.333914,0.245771,0.278886,0.325486,59.67,296.345714,297.521429,...,18,4,169,False,False,False,False,False,False,1276819200
1455,iq,2010,25,0.298186,0.232971,0.274214,0.315757,63.22,298.097143,299.835714,...,25,4,176,False,False,False,False,False,False,1277424000


In [124]:
df_sj, y_sj, nas_sj = proc_df(train_sj, 'total_cases')

In [125]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 182  # same as Kaggle's test set size
n_trn = len(train_sj)-n_valid
raw_train_sj, raw_valid_sj = split_vals(train_sj, n_trn)
X_train_sj, X_valid_sj = split_vals(df_sj, n_trn)
y_train_sj, y_valid_sj = split_vals(y_sj, n_trn)

X_train_sj.shape, y_train_sj.shape, X_valid_sj.shape, y_valid_sj.shape

((754, 54), (754,), (182, 54), (182,))

In [126]:
df_iq, y_iq, nas_iq = proc_df(train_iq, 'total_cases')

In [127]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 110  # same as Kaggle's test set size
n_trn = len(train_iq)-n_valid
raw_train_iq, raw_valid_iq = split_vals(train_iq, n_trn)
X_train_iq, X_valid_iq = split_vals(df_iq, n_trn)
y_train_iq, y_valid_iq = split_vals(y_iq, n_trn)

X_train_iq.shape, y_train_iq.shape, X_valid_iq.shape, y_valid_iq.shape

((410, 54), (410,), (110, 54), (110,))

In [167]:
X_train = pd.concat([X_train_sj, X_train_iq]);
X_train.reset_index(drop = True, inplace = True)
X_train.shape

(1164, 54)

In [174]:
X_valid = pd.concat([X_valid_sj, X_valid_iq]); 
X_valid.reset_index(drop = True, inplace = True)
X_valid.shape

(292, 54)

In [162]:
X_train.to_feather('tmp/dengue-Xtrain')
X_valid.to_feather('tmp/dengue-Xvalid')

In [177]:
y_train = np.concatenate([y_train_sj, y_train_iq])
y_train = pd.DataFrame(data = y_train)
y_train.shape

(1164, 1)

In [181]:
y_valid = np.concatenate([y_valid_sj, y_valid_iq])
y_valid = pd.DataFrame(data = y_valid)
y_valid.shape

(292, 1)

In [183]:
# y_train.to_feather('tmp/dengue-ytrain')
# y_valid.to_feather('tmp/dengue-yvalid')

Making the dependent variable into a feather will be a lot more convenient, but I will get to that later. 

Validation set may have to be chanegd as well. 

## Rerunning a random forest 

In [163]:
X_train = pd.read_feather('tmp/dengue-Xtrain')
X_valid = pd.read_feather('tmp/dengue-Xvalid')

In [182]:
m2 = RandomForestRegressor(n_jobs=-1)
%time m2.fit(X_train, y_train)
print_score(m2)

/Users/chanmunfai/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


CPU times: user 1.99 s, sys: 28 ms, total: 2.02 s
Wall time: 765 ms
['mae train: ', 3.7869158075601375, 'mae val: ', 13.431335616438355, 'R square train:', 0.9656169217649994, 'R square val:', 0.15209596012010884]


Here, we see that our performance has improved by looking at both cities. However, there is a strong time-based dependency which our model is unable to deal with. 

This model with absolutely no tweaking gave us a MAE of 29.5793 which is a pretty bad rank of 2879/9207 competitors. 

In [284]:
m3 = RandomForestRegressor(n_estimators = 1600 , max_features = 'log2', min_samples_leaf =2, oob_score = True,  n_jobs=-1)
%time m3.fit(X_train, y_train)
print_score(m3)

/Users/chanmunfai/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


CPU times: user 5.25 s, sys: 434 ms, total: 5.69 s
Wall time: 3.29 s
['mae train: ', 7.024826683279827, 'mae val: ', 14.062645360270384, 'R square train:', 0.9102391877823006, 'R square val:', 0.22031864797629286, 0.7170923778279861]


In [293]:
m4 = RandomForestRegressor(n_estimators = 8000 , max_features = 'log2', min_samples_leaf =2, oob_score = True,  n_jobs=-1)
%time m4.fit(X_train, y_train)
print_score(m4)

/Users/chanmunfai/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


CPU times: user 28 s, sys: 2.38 s, total: 30.3 s
Wall time: 19.3 s
['mae train: ', 7.0255896668690925, 'mae val: ', 14.104636131763833, 'R square train:', 0.9105709016401224, 'R square val:', 0.22233541061603357, 0.7198413834204558]


We submitted this and got an MAE of 29.5793. It seems like we can't do much better at this model by simply running it as many times as we can. 

This is slightly worse than the baseline score, which isn't that bad given that we used absolutely no techniques in accounting for the time series, and that random forests are generally bad at extrapolating outside the time zone. 

We will then go on to experiment more with pre and post processing methods to deal with the time series. 

Useful link : https://www.statworx.com/at/blog/time-series-forecasting-with-random-forest/


Baseline model : https://www.drivendata.co/blog/dengue-benchmark/

## Submission Script 

In [185]:
samp_sub = pd.read_csv("/Users/chanmunfai/fastai/courses/ml1/Dengue/submission_format.csv", low_memory = False)

In [186]:
samp_sub.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,0
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,0
4,sj,2008,22,0


#### Making the test set the same as the training set 

In [198]:
test = pd.read_csv("/Users/chanmunfai/fastai/courses/ml1/Dengue/dengue_features_test.csv", low_memory = False)

In [194]:
test.shape, train.shape

((416, 24), (1456, 35))

In [199]:
add_datepart(test, 'week_start_date')
test.drop(['week_start_Year', 'week_start_Week'], axis =1, inplace = True)
train_cats(test)

In [205]:
test.shape

((416, 34), (1164, 54))

In [201]:
display_all(test.isnull().sum().sort_index()/len(test))

city                                     0.000000
ndvi_ne                                  0.103365
ndvi_nw                                  0.026442
ndvi_se                                  0.002404
ndvi_sw                                  0.002404
precipitation_amt_mm                     0.004808
reanalysis_air_temp_k                    0.004808
reanalysis_avg_temp_k                    0.004808
reanalysis_dew_point_temp_k              0.004808
reanalysis_max_air_temp_k                0.004808
reanalysis_min_air_temp_k                0.004808
reanalysis_precip_amt_kg_per_m2          0.004808
reanalysis_relative_humidity_percent     0.004808
reanalysis_sat_precip_amt_mm             0.004808
reanalysis_specific_humidity_g_per_kg    0.004808
reanalysis_tdtr_k                        0.004808
station_avg_temp_c                       0.028846
station_diur_temp_rng_c                  0.028846
station_max_temp_c                       0.007212
station_min_temp_c                       0.021635


We also have missing values for the test set, and we need to impute them accordingly with the same imputations as the train set. 

In [203]:
test.dtypes

city                                     category
year                                        int64
weekofyear                                  int64
ndvi_ne                                   float64
ndvi_nw                                   float64
ndvi_se                                   float64
ndvi_sw                                   float64
precipitation_amt_mm                      float64
reanalysis_air_temp_k                     float64
reanalysis_avg_temp_k                     float64
reanalysis_dew_point_temp_k               float64
reanalysis_max_air_temp_k                 float64
reanalysis_min_air_temp_k                 float64
reanalysis_precip_amt_kg_per_m2           float64
reanalysis_relative_humidity_percent      float64
reanalysis_sat_precip_amt_mm              float64
reanalysis_specific_humidity_g_per_kg     float64
reanalysis_tdtr_k                         float64
station_avg_temp_c                        float64
station_diur_temp_rng_c                   float64


In [207]:
X_test, _, nas = proc_df(test, na_dict=nas)

In [210]:
X_test.shape, X_train.shape

((416, 54), (1164, 54))

#### Submission 1 

In [211]:
pred_cases1 = m2.predict(X_test)

In [212]:
submission1 = pd.DataFrame()
submission1['city'] = test['city']
submission1['year'] = test['year']
submission1['weekofyear'] = test['weekofyear']

In [253]:
pred = pred_cases1.astype(int, copy = True);

In [254]:
submission1['total_cases']= pred

submission1.head()

In [256]:
submission1.to_csv('dengAI_subm1.csv', index=False)

### Submission 2 

In [285]:
pred_cases2 = m3.predict(X_test)

In [286]:
submission2 = pd.DataFrame()
submission2['city'] = test['city']
submission2['year'] = test['year']
submission2['weekofyear'] = test['weekofyear']

In [287]:
pred2 = pred_cases2.astype(int, copy = True);

In [290]:
submission2['total_cases']= pred2

In [291]:
submission2.to_csv('dengAI_subm2.csv', index=False)

### Submission 3 

In [294]:
pred_cases3 = m4.predict(X_test)

In [295]:
submission3 = pd.DataFrame()
submission3['city'] = test['city']
submission3['year'] = test['year']
submission3['weekofyear'] = test['weekofyear']

In [296]:
pred3 = pred_cases3.astype(int, copy = True);

In [297]:
submission3['total_cases']= pred3

In [298]:
submission3.to_csv('dengAI_subm3.csv', index=False)